In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime, timedelta
import seaborn as sn
import time

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Диплом/data_for_science (1).csv', sep=';')

In [ ]:
df['idx'].unique().shape

(1125982,)

In [ ]:
# Drop all incorrect get_date_at formats
df = df[df['get_date_at'].apply(lambda x: x[0]) == '2']

In [ ]:
df['get_date_at'] = pd.to_datetime(df['get_date_at'])

In [ ]:
df['dis_year'] = pd.to_datetime(df['get_date_at']).dt.year
df = df[df['dis_year'] >= 2020]
df = df[df['get_date_at'].dt.month >= 4]
df = df.sort_values('get_date_at')

In [ ]:
df[df['mkb10_code'] == 'U07.1']['idx'].unique().shape

(56006,)

In [ ]:
# Create a zero-filled 'test_day' column
df['test_day'] = np.zeros(df.shape[0]).astype(int)

In [ ]:
# Drop all records that belongs to patients with only one record
df = df[df['idx'].isin(df['idx'].value_counts()[df['idx'].value_counts() != 1].index)].reset_index().drop(['index'], 1)

In [ ]:
# Adding a IgM_target vector that contains True if test IgM > 1, False otherwise
df['IgM_target'] = df['IgM'] > 1

In [ ]:
# Adding a IgG_target vector that contains True if test IgG > 10, False otherwise
df['IgG_target'] = df['IgG'] > 10

In [ ]:
# Drop records whos' idx user don't have IgG > 10 anytime
df = df[df['idx'].isin(df.groupby(['idx']).sum()[df.groupby(['idx']).sum()['IgG_target'] > 0].reset_index()['idx'])]

In [ ]:
people = pd.DataFrame(np.array([df['idx'].unique().astype(int), np.zeros(df['idx'].unique().size).astype(int)]).T, columns=['idx', 'day_number'])

In [ ]:
def SetTestNumber(idx):
    x = df[df['idx'] == idx]
    return (min(x[x['IgG_target']]['get_date_at']) - min(np.hstack((x[x['IgG_target']]['get_date_at'], x[x['IgM_target']]['get_date_at'])))).days

VSetTestNumber = np.vectorize(SetTestNumber)

In [ ]:
people['day_number'] = VSetTestNumber(people['idx'])

In [ ]:
#plt.hist(people['day_number'], bins=max(people['day_number']))

In [ ]:
people.to_excel('/content/drive/MyDrive/Диплом/Immun.xlsx')

Выделяем тех, у кого IgM >1, подтвержденный ковид


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Диплом/data_for_science (1).csv', sep=';')

In [ ]:
df = df[(df['mkb10_code'] == 'U07.1')]

In [ ]:
df = df[df['IgM'] > 1] 

In [ ]:
Acute_illness = df['idx'].unique()

In [ ]:
Acute_illness.shape

(6030,)

In [ ]:
pd.DataFrame(Acute_illness, columns=['idx']).to_excel('/content/drive/MyDrive/Диплом/Acute_illness.xlsx')

Выделяем тех, у кого имунная память

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Диплом/data_for_science (1).csv', sep=';')

In [ ]:
df = df[(df['mkb10_code'] == 'U07.1')]

In [ ]:
df = df[df['idx'].isin(Acute_illness)]

In [ ]:
df = df[(df['IgG']>10) & (df['IgM'] < 1)] 

In [ ]:
Immun_memory = df['idx'].unique()

In [ ]:
pd.DataFrame(Immun_memory, columns=['idx']).to_excel('/content/drive/MyDrive/Диплом/Immun_memory.xlsx')

In [ ]:
Immun_memory.shape

(188,)

Нет иммунной памяти после перенесенного заболевания

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Диплом/data_for_science (1).csv', sep=';')

In [ ]:
df = df[(df['mkb10_code'] == 'U07.1')]

In [ ]:
df = df[df['idx'].isin(Acute_illness)]

In [ ]:
df = df[~df['idx'].isin(Immun_memory)]

In [ ]:
df['idx'].unique().shape

(5842,)

In [ ]:
No_immun_memory = df[(df['IgG'] < 10) & (df['IgM']  < 1)]['idx'].unique()
Lost = df['idx'].unique()[~np.isin(df['idx'].unique(), No_immun_memory)]

In [ ]:
No_immun_memory.shape, Lost.shape, Acute_illness.shape

((200,), (5642,), (6030,))

In [ ]:
pd.DataFrame(No_immun_memory, columns=['idx']).to_excel('/content/drive/MyDrive/Диплом/No_immun_memory.xlsx')
pd.DataFrame(Lost, columns=['idx']).to_excel('/content/drive/MyDrive/Диплом/Lost.xlsx')

Выделяем только тех, у кого был контакт, начиная с 13 мая

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Диплом/data_for_science (1).csv', sep=';')

In [ ]:
df = df[(df['mkb10_code'] == 'Z20.8') & (df['dis_date'] >= '2020-05-13')]

In [ ]:
df['idx'].unique().shape

(194115,)

In [ ]:
Contact = df['idx'].unique()

In [ ]:
pd.DataFrame(df['idx'].unique(), columns=['idx']).to_excel('/content/drive/MyDrive/Диплом/Contact.xlsx')


Выделяем только тех, у кого подтвержденный ковид

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Диплом/data_for_science (1).csv', sep=';')

In [ ]:
df = df[df['idx'].isin(Contact)]

In [ ]:
df = df[(df['mkb10_code'] == 'U07.1') & (df['dis_date'] >= '2020-05-13')]

In [ ]:
df.shape

(23162, 17)

In [ ]:
pd.DataFrame(df['idx'].unique(), columns=['idx']).to_excel('/content/drive/MyDrive/Диплом/Illness.xlsx')

In [ ]:
Illness = df['idx'].unique()

In [ ]:
Illness

array([    223,     283,     314, ..., 1266087, 1267563, 1268384])

Acute_illness, все из списка Illness, у кого есть IgM > 1

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Диплом/data_for_science (1).csv', sep=';')

In [ ]:
df = df[df['idx'].isin(Illness)]

In [ ]:
df = df[df['IgM'] > 1] 

In [ ]:
df = df[(df['mkb10_code'] == 'U07.1')]

In [ ]:
pd.DataFrame(df['idx'].unique(), columns=['idx']).to_excel('/content/drive/MyDrive/Диплом/Acute_illness.xlsx')

In [ ]:
Acute_illness =  df['idx'].unique()

Immun_memory, все из списка Acute_illness, у кого тест на имунную память позже положителен, чем на острую инфекцию

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Диплом/data_for_science (1).csv', sep=';')

In [ ]:
df = df[df['idx'].isin(Acute_illness)]

In [ ]:
df = df[(df['mkb10_code'] == 'U07.1')]

In [ ]:
Immun_memory = pd.DataFrame(Acute_illness, columns=['idx'])

In [ ]:
def ImmGM ( idx ):
  return min(np.hstack([df[(df['idx'] == idx) & (df['IgM'] < 1) & (df['IgG'] > 10)]['get_date_at'], '2050-01-01']))

In [ ]:
def ImmM ( idx ):
  return min(df[(df['idx'] == idx) & (df['IgM'] > 1)]['get_date_at'])


In [ ]:
np.vectorize(ImmM)(Immun_memory['idx'])

array(['2020-06-25', '2020-05-21', '2020-06-05', ..., '2020-07-02',
       '2020-07-02', '2020-07-03'], dtype='<U10')

In [ ]:
#Когда тест IgM был > 1
Immun_memory['test_day_M'] = np.vectorize(ImmM)(Immun_memory['idx'])

In [ ]:
#Когда тест IgM < 1 & IgG > 10
Immun_memory['test_day_G_M'] = np.vectorize(ImmGM)(Immun_memory['idx'])

In [ ]:
Immun_memory = Immun_memory[Immun_memory['test_day_G_M'] != '2050-01-01']

In [ ]:
Immun_memory = Immun_memory[Immun_memory['test_day_G_M'] > Immun_memory['test_day_M']]
#Immun_memory

In [ ]:
pd.DataFrame(Immun_memory['idx'].unique(), columns=['idx']).to_excel('/content/drive/MyDrive/Диплом/Immun_memory.xlsx')

In [ ]:
Immun_memory['idx'].unique().shape

(32,)

No_immun_memory, нет иммунной памяти после перенесенного заболевания
 *(второй или теретий диагноз  IgM<1, IgG<10)*

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Диплом/data_for_science (1).csv', sep=';')

In [ ]:
df = df[df['idx'].isin(Acute_illness)]

In [ ]:
df = df[(df['mkb10_code'] == 'U07.1')]

In [ ]:
def ImmM ( idx ):
  return min(df[(df['idx'] == idx) & (df['IgM'] > 1)]['get_date_at'])


In [ ]:
np.vectorize(ImmM)(Immun_memory['idx'])

array(['2020-06-22', '2020-05-27', '2020-05-20', '2020-06-04',
       '2020-06-08', '2020-06-16', '2020-05-23', '2020-05-25',
       '2020-06-07', '2020-06-04', '2020-06-03', '2020-06-03',
       '2020-05-26', '2020-06-20', '2020-06-03', '2020-05-28',
       '2020-06-07', '2020-05-28', '2020-06-03', '2020-06-03',
       '2020-06-03', '2020-06-05', '2020-06-04', '2020-06-03',
       '2020-06-03', '2020-06-04', '2020-06-03', '2020-06-04',
       '2020-06-04', '2020-06-08', '2020-06-05', '2020-06-12'],
      dtype='<U10')

In [ ]:
No_immun_memory = pd.DataFrame(Acute_illness, columns=['idx'])

In [ ]:
#Когда тест IgM был > 1
No_immun_memory['test_day_M'] = np.vectorize(ImmM)(No_immun_memory['idx'])

In [ ]:
def ImmGM_no ( idx ):
  return min(np.hstack([df[(df['idx'] == idx) & (df['IgM'] < 1) & (df['IgG'] < 10)]['get_date_at'], '2050-01-01']))

In [ ]:
np.vectorize(ImmGM_no)(No_immun_memory['idx'])

array(['2050-01-01', '2050-01-01', '2050-01-01', ..., '2050-01-01',
       '2050-01-01', '2050-01-01'], dtype='<U10')

In [ ]:
#Когда тест IgM < 1 & IgG > 10
No_immun_memory['day_M_G'] = np.vectorize(ImmGM_no)(No_immun_memory['idx'])

In [ ]:
No_immun_memory = No_immun_memory[No_immun_memory['day_M_G'] != '2050-01-01']

In [ ]:
No_immun_memory = No_immun_memory[No_immun_memory['day_M_G'] > No_immun_memory['test_day_M']]

In [ ]:
pd.DataFrame(No_immun_memory['idx'].unique(), columns=['idx']).to_excel('/content/drive/MyDrive/Диплом/No_immun_memory.xlsx')

Lost, группа потерянных из наблюдения

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Диплом/data_for_science (1).csv', sep=';')

In [ ]:
#из Acute illness те, кто выходят из наблюдения

df = df[df['idx'].isin(Acute_illness)]
df = df[~df['idx'].isin(Immun_memory['idx'])]
df = df[~df['idx'].isin(No_immun_memory['idx'])]

In [ ]:
Lost = pd.DataFrame(df, columns=['idx'])

In [ ]:
pd.DataFrame(Lost['idx'].unique(), columns=['idx']).to_excel('/content/drive/MyDrive/Диплом/Lost.xlsx')